In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import pandas as pd
import numpy as np
import torch.optim as optim

In [2]:
import os

In [71]:
import time

In [3]:
os.listdir()

['未命名.ipynb',
 '.ipynb_checkpoints',
 'Mine.ipynb',
 'sample_submission.csv',
 'test.csv',
 'Reference.ipynb',
 'torch.ipynb',
 'train.csv']

In [4]:
train = pd.read_csv('train.csv', dtype=np.float32)
test = pd.read_csv('test.csv', dtype=np.float32)

In [5]:
train.head(2)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
test.head(2)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
feature = train.iloc[:, 1:].values
target = train.iloc[:,0].values

In [19]:
feature.shape

(42000, 784)

In [20]:
target.shape

(42000,)

In [21]:
f_tensor = torch.from_numpy(feature)
t_tensor = torch.from_numpy(target)

In [23]:
feature = f_tensor
target = t_tensor.long()

In [25]:
feature.shape

torch.Size([42000, 784])

In [27]:
feature = feature.reshape(-1,28,28)

In [28]:
feature.shape

torch.Size([42000, 28, 28])

In [30]:
feature = feature.unsqueeze(dim=1)

In [34]:
feature.shape # dim=0时表示图片数量

torch.Size([42000, 1, 28, 28])

In [36]:
train_tensor = TensorDataset(feature, target) # 封装进TensorData的时候，一定要事先将两个转换为tensor

In [38]:
train_loader = DataLoader(train_tensor, batch_size=100, shuffle=True)

In [58]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        
        self.out = nn.Linear(in_features=60, out_features=10)
    
    def forward(self, t):
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2) # 会自动调用魔法方法call
        
        t = F.relu(self.fc1(t.reshape(-1,12*4*4)))
        
        t = F.relu(self.fc2(t))
        
        return self.out(t)

In [94]:
device1 = torch.device('cpu')
network = Network().to(device1)

In [105]:
optimizer = optim.Adam(network.parameters(), lr=0.01) # 选择合适的优化算法

In [79]:
def get_correct_num(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [81]:
for epoch in range(10):
    total_loss = 0
    total_correct = 0
    time1 = time.time()
    for feature, label in train_loader:
        preds = network(feature)
        
        loss = F.cross_entropy(preds, label)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        total_loss += loss
        
        total_correct += get_correct_num(preds, label)
    time2 = time.time()
    print(f'epoch:{epoch}, time:{time2-time1}')
    print(f'epoch:{epoch}, acc:{total_correct/train.shape[0]}, loss:{total_loss}')

epoch:0, time:4.491867303848267
epoch:0, acc:0.9473333333333334, loss:93.97937774658203
epoch:1, time:4.5379931926727295
epoch:1, acc:0.9506666666666667, loss:88.90043640136719
epoch:2, time:4.806806325912476
epoch:2, acc:0.9497857142857142, loss:89.56981658935547
epoch:3, time:4.656897068023682
epoch:3, acc:0.9528333333333333, loss:86.91255187988281
epoch:4, time:4.437143802642822
epoch:4, acc:0.9563095238095238, loss:78.534423828125
epoch:5, time:4.573753118515015
epoch:5, acc:0.9498095238095238, loss:90.45974731445312
epoch:6, time:4.485816955566406
epoch:6, acc:0.9456190476190476, loss:105.58292388916016
epoch:7, time:4.447885751724243
epoch:7, acc:0.9503809523809523, loss:91.49393463134766
epoch:8, time:4.512714147567749
epoch:8, acc:0.9471428571428572, loss:97.46285247802734
epoch:9, time:4.8309149742126465
epoch:9, acc:0.9472142857142857, loss:97.20996856689453


In [112]:
device2 = torch.device('cuda:0')

In [113]:
network = Network()
network.to(device2)

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)

In [154]:
optimizer = optim.Adam(network.parameters(), lr=0.000001) # 选择合适的优化算法

In [155]:
for epoch in range(10):
    total_loss = 0
    total_correct = 0
    
    time1 = time.time()
    
    for feature, label in train_loader:
        feature = feature.to(device2)
        label = label.to(device2)
        
        preds = network(feature)
        
        loss = F.cross_entropy(preds, label)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        total_loss += loss
        
        total_correct += get_correct_num(preds, label)
    time2 = time.time()
    print(f'epoch:{epoch}, time:{time2-time1}')
    print(f'epoch:{epoch}, acc:{total_correct/train.shape[0]}, loss:{total_loss}')

epoch:0, time:2.7753853797912598
epoch:0, acc:1.0, loss:0.011738086119294167
epoch:1, time:2.7438390254974365
epoch:1, acc:1.0, loss:0.011443897150456905
epoch:2, time:2.726247549057007
epoch:2, acc:1.0, loss:0.01123904436826706
epoch:3, time:2.854438543319702
epoch:3, acc:1.0, loss:0.011054527014493942
epoch:4, time:2.789785385131836
epoch:4, acc:1.0, loss:0.010902632027864456
epoch:5, time:2.822584390640259
epoch:5, acc:1.0, loss:0.010732005350291729
epoch:6, time:2.8636138439178467
epoch:6, acc:1.0, loss:0.01057131215929985
epoch:7, time:2.9709086418151855
epoch:7, acc:1.0, loss:0.010416884906589985
epoch:8, time:2.9457509517669678
epoch:8, acc:1.0, loss:0.010273055173456669
epoch:9, time:2.7179958820343018
epoch:9, acc:1.0, loss:0.010118963196873665


In [118]:
# 在实验的过程中，学习率一直设置为0.01，先跑10个epoch，loss一直在降低
# 然后一直使用这个学习率，再跑10个epoch，loss在不断升高
# 再跑10个epoch，loss升的非常厉害
# 为什么会出现这样的情况？

In [119]:
# 解决办法：每次学习训练，都调低学习率，越到后面，学习率应该越低

In [156]:
test_tensor = torch.from_numpy(test.values)

In [157]:
test.shape

(28000, 784)

In [158]:
test_tensor = test_tensor.reshape(-1, 28, 28)

In [159]:
test_tensor.shape

torch.Size([28000, 28, 28])

In [160]:
test_tensor = test_tensor.unsqueeze(dim=1)

In [161]:
test_tensor.shape

torch.Size([28000, 1, 28, 28])

In [162]:
test_tensor = test_tensor.to(device2)

In [163]:
res = network(test_tensor)

In [164]:
res.shape

torch.Size([28000, 10])

In [165]:
res

tensor([[ -9.5048,  -2.7488,  44.3806,  ...,  -2.3140,   7.2946,  -8.1986],
        [ 29.4733, -23.1844, -10.1062,  ...,  -8.3819,  -9.3499,  -4.1980],
        [ -6.7087, -19.4331, -11.3997,  ..., -13.9866,   3.0919,  22.4174],
        ...,
        [-24.6729, -11.2181, -12.5951,  ...,  -9.3148,  -2.3586,  -4.3807],
        [-13.1518,  -9.4155, -14.2537,  ...,  -2.1433, -12.8389,  32.4885],
        [ -7.3835,  -6.7860,  54.7024,  ...,  -3.3791,   8.1150,  -4.9464]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [166]:
res = res.argmax(dim=1)

In [167]:
res.shape

torch.Size([28000])

In [168]:
res

tensor([2, 0, 9,  ..., 3, 9, 2], device='cuda:0')

In [169]:
res = res.to(device1)

In [170]:
res

tensor([2, 0, 9,  ..., 3, 9, 2])

In [172]:
df = pd.DataFrame({'imageId':range(1,len(res)+1),'Label':res})

In [173]:
df

,imageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [174]:
df.to_csv('sample_submission.csv', index=False)